In [7]:
# Import Libraries
import random
import time
import copy
from IPython.display import clear_output

In [8]:
# Question 1 methods :-
# ChessBoard

def ChessBoardSetup():
  global board, pieces
  board = [['.']*8 for i in range(8)]
  pieces = ['r', 't', 'b', 'q', 'k']

  for i in range(3):
    board[0][i] = pieces[i]
    board[0][7 - i] = pieces[i]
    board[7][i] = pieces[i].upper()
    board[7][7 - i] = pieces[i].upper()

  for i in range(2):
    board[0][3 + i] = pieces[3 + i]
    board[7][3 + i] = pieces[3 + i].upper()

  for i in range(8):
    board[1][i] = 'p'
    board[6][i] = 'P'


def DrawBoard():
  for i in range(8):
    for j in range(8):
      print(board[i][j], end=" ")
    print()
  print()


def MovePiece(from_square, to_square):
  global board
  (xi, yi) = from_square
  (xf, yf) = to_square
  newBoard = copy.deepcopy(board)
  newBoard[xf][yf] = newBoard[xi][yi]
  newBoard[xi][yi] = '.'
  return copy.deepcopy(newBoard)


In [9]:
# Question 2 methods :-
# ChessRules

def IsRookClearPath(from_square, to_square):
  (xi, yi) = from_square
  (xf, yf) = to_square
  if xi == xf:
    for i in range(min(yi, yf) + 1, max(yi, yf)):
      if board[xi][i] != '.':
        return False
    return True
  elif yi == yf:
    for i in range(min(xi, xf) + 1, max(xi, xf)):
      if board[i][yi] != '.':
        return False
    return True
  return False


def IsBishopClearPath(from_square, to_square):
  (xi, yi) = from_square
  (xf, yf) = to_square
  xt = xf - xi
  yt = yf - yi
  if abs(xt) == abs(yt):
    x_dir = -1 if xf < xi else 1
    y_dir = -1 if yf < yi else 1
    c = 0
    while xi != xf and yi != yf:
      if c > 0 and board[xi][yi] != '.':
        return False
      c += 1
      xi += x_dir
      yi += y_dir
    if xi == xf and yi == yf:
      return True
  return False


def IsClearPath(from_square, to_square):
  (xi, yi) = from_square
  (xf, yf) = to_square
  piece = board[xi][yi]
  if piece == 'R' or piece == 'r':
    return IsRookClearPath(from_square, to_square)
  elif piece == 'B' or piece == 'b':
    return IsBishopClearPath(from_square, to_square)
  elif piece == 'Q' or piece == 'q':
    return IsRookClearPath(from_square, to_square) or IsBishopClearPath(from_square, to_square)


def isOpponentPiece(from_piece, to_piece):
  return (from_piece.isupper() ^ to_piece.isupper())


def IsMoveLegal(from_square, to_square):
    (xi, yi) = from_square
    (xf, yf) = to_square
    from_piece = board[xi][yi]
    to_piece = board[xf][yf]

    if from_square == to_square:
      return False

    if from_piece == 'P':
      # move forward
      if xf == xi - 1:
        if to_piece == '.' and yi == yf:
          return True
      # move 2 steps forward
      if xi == 6 and xf == xi - 2:
        if board[xi - 1][yi] == '.' and to_piece == '.' and yi == yf:
          return True
        else:
          return False
      # move diagonally
      if xf == xi - 1:
        if abs(yf - yi) == 1 and to_piece != '.' and to_piece.islower():
          return True
      return False

    elif from_piece == 'p':
      # move forward
      if xf == xi + 1:
        if to_piece == '.' and yi == yf:
          return True
      # move 2 steps forward
      if xi == 1 and xf == xi + 2:
        if board[xi + 1][yi] == '.' and to_piece == '.' and yi == yf:
          return True
        else:
          return False
      # move diagonally
      if xf == xi + 1:
        if abs(yf - yi) == 1 and board[xf][yf] != '.' and board[xf][yf].isupper():
          return True
      return False

    elif from_piece in ['R', 'B', 'Q']:
      if to_piece == '.' or to_piece.islower():
        return IsClearPath(from_square, to_square)

    elif from_piece in ['r', 'b', 'q']:
      if to_piece == '.' or to_piece.isupper():
        return IsClearPath(from_square, to_square)

    elif from_piece in ['t', 'T']:
      row_diff = abs(xi - xf)
      col_diff = abs(yi - yf)
      if (to_piece == '.' or isOpponentPiece(from_piece, to_piece)) and (row_diff == 1 and col_diff == 2) or (row_diff == 2 and col_diff == 1):
        return True

    elif from_piece in ['k', 'K']:
      row_diff = abs(xi - xf)
      col_diff = abs(yi - yf)
      if (to_piece == '.' or isOpponentPiece(from_piece, to_piece)) and (row_diff <= 1 and col_diff <= 1):
        return True

    return False


def GetListOfLegalMoves(player, from_square):
    legal_moves = []
    for i in range(8):
      for j in range(8):
        to_square = (i, j)
        if IsMoveLegal(from_square, to_square):
          if not DoesMovePutPlayerInCheck(player, from_square, to_square):
            legal_moves.append(to_square)
    return legal_moves


def isPieceBelongsToPlayer(player, piece):
  return (piece != '.' and ((player == 1 and piece.isupper()) or (player == 0 and piece.islower())))


def GetPiecesWithLegalMoves(player):
    pieces_with_legal_moves = []
    for i in range(8):
      for j in range(8):
        from_square = (i, j)
        if isPieceBelongsToPlayer(player, board[i][j]):
          legal_moves = GetListOfLegalMoves(player, from_square)
          if len(legal_moves) > 0:
            pieces_with_legal_moves.append(from_square)
    return pieces_with_legal_moves


def IsCheckmate(player):
    legal_moves = GetPiecesWithLegalMoves(player)
    return (len(legal_moves) == 0)


def IsInCheck(player):
    location = (0, 0)
    for i in range(8):
      for j in range(8):
        if board[i][j].lower() == 'k' and isPieceBelongsToPlayer(player, board[i][j]):
          location = (i, j)
          break

    for i in range(8):
      for j in range(8):
        from_square = (i, j)
        opponent = 0 if player else 1
        if isPieceBelongsToPlayer(opponent, board[i][j]):
          if IsMoveLegal(from_square, location):
            return True

    return False


def DoesMovePutPlayerInCheck(player, from_square, to_square):
  global board
  pastBoard = copy.deepcopy(board)
  board = MovePiece(from_square, to_square)
  check = IsInCheck(player)
  board = copy.deepcopy(pastBoard)
  return check


In [10]:
# Question 3 methods :-
# RandomAI

def GetRandomMove(player):
  from_legal_moves = GetPiecesWithLegalMoves(player)
  from_square = random.choice(from_legal_moves)
  to_legal_moves = GetListOfLegalMoves(player, from_square)
  to_square = random.choice(to_legal_moves)
  return from_square, to_square


In [11]:
# Question 4 methods :-
# MinMaxAI

def evl():
    score = 0
    scoring = {'p': -1, 't': -3, 'b': -3, 'r': -5, 'q': -9, 'k': 0,
               'P': 1, 'T': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 0, '.': 0}
    for i in range(8):
      for j in range(8):
        score += scoring[board[i][j]]

    return score


def GetMinMaxMove(player):
    global board
    curr_board_score = evl()

    pieces = GetPiecesWithLegalMoves(player)
    player_scores = []
    player_pos = []

    for from_square in pieces:
      moves = GetListOfLegalMoves(player, from_square)
      for move in moves:
        # move temporarily for player
        prevBoard = copy.deepcopy(board)
        board = MovePiece(from_square, move)

        enemy_player = 0 if player else 1
        enemy_pieces = GetPiecesWithLegalMoves(enemy_player)
        enemy_scores = []
        enemy_pos = []

        for from_enemy_square in enemy_pieces:
          enemy_moves = GetListOfLegalMoves(enemy_player, from_enemy_square)
          for enemy_move in enemy_moves:
            # move temporarily for enemy
            prevEnemyBoard = copy.deepcopy(board)
            board = MovePiece(from_enemy_square, enemy_move)

            enemy_scores.append(evl())
            enemy_pos.append((from_enemy_square, enemy_move))

            # undo the enemy move
            board = copy.deepcopy(prevEnemyBoard)

        # if enemy player is having white, maximize score
        if enemy_player:
          enemy_best_move = enemy_pos[enemy_scores.index(max(enemy_scores))]
          enemy_best_score = max(enemy_scores)
        else:
          enemy_best_move = enemy_pos[enemy_scores.index(min(enemy_scores))]
          enemy_best_score = min(enemy_scores)

        player_pos.append(enemy_best_move)
        player_scores.append(enemy_best_score)

        # undo the player move
        board = copy.deepcopy(prevBoard)

    if player:
      player_best_move = player_pos[player_scores.index(max(player_scores))]
      player_best_score = max(player_scores)
    else:
      player_best_move = player_pos[player_scores.index(min(player_scores))]
      player_best_score = min(player_scores)

    if (player and player_best_score > curr_board_score) or ((not player) and player_best_score < curr_board_score):
      return player_best_move

    return GetRandomMove(player)


In [ ]:
# Game Setup & Main Loop

# white - uppercase - player 1
# black - lowercase - player 0
global board

ChessBoardSetup()
player = 1
turns = 0
N = 100

while (not IsCheckmate(player)) and turns < N:
    if player:
      from_square, to_square = GetRandomMove(player)
    else:
      from_square, to_square = GetMinMaxMove(player)

    board = MovePiece(from_square, to_square)
    turns += 1
    player ^= 1

    DrawBoard()
    time.sleep(0.5)

if turns == N:
  print('Stalemate')
else:
  print(f'Checkmate player {player}')


r t b q k b t r 
p p p p p p p p 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . T . . 
P P P P P P P P 
R T B Q K B . R 

r t b q k b t r 
p p p p p p . p 
. . . . . . p . 
. . . . . . . . 
. . . . . . . . 
. . . . . T . . 
P P P P P P P P 
R T B Q K B . R 

r t b q k b t r 
p p p p p p . p 
. . . . . . p . 
. . . . . . . . 
P . . . . . . . 
. . . . . T . . 
. P P P P P P P 
R T B Q K B . R 

r t b q k b t r 
p p p . p p . p 
. . . p . . p . 
. . . . . . . . 
P . . . . . . . 
. . . . . T . . 
. P P P P P P P 
R T B Q K B . R 

r t b q k b t r 
p p p . p p . p 
. . . p . . p . 
. . . . . . . . 
P . . P . . . . 
. . . . . T . . 
. P P . P P P P 
R T B Q K B . R 

r t b q k b . r 
p p p . p p . p 
. . . p . . p t 
. . . . . . . . 
P . . P . . . . 
. . . . . T . . 
. P P . P P P P 
R T B Q K B . R 

r t b q k b . r 
p p p . p p . p 
. . . p . . p t 
. . . . . . . . 
P . . P . . . . 
. P . . . T . . 
. . P . P P P P 
R T B Q K B . R 

r t b q k b . r 
p p p . p . . p 
. . . p